In [6]:
%pip install selenium parsel

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\anfiniti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Import Service untuk mendefinisikan path
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd

# Buat list untuk menyimpan data
data = []

# Tentukan path ke Chromedriver
chromedriver_path = './chromedriver.exe'  # Ubah sesuai lokasi file Anda

# Buat objek Service dengan path driver
service = Service(chromedriver_path)

# Opsi untuk membuka Chrome dalam bahasa Indonesia dan menonaktifkan pemuatan gambar
options = webdriver.ChromeOptions()
options.add_argument("--lang=id")
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

# Inisialisasi webdriver dengan opsi
driver = webdriver.Chrome(service=service, options=options)

# Buka halaman web
driver.get("https://maps.app.goo.gl/fMaHmC9xszQrcrHB6")

# Tunggu elemen tombol "Lihat Semua Ulasan" muncul
try:
    see_all_reviews_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//button[contains(@aria-label, "Ulasan untuk Universitas Stikubank (Unisbank) Semarang")]'))
    )
    see_all_reviews_button.click()
except Exception as e:
    print("Tombol 'Lihat Semua Ulasan' tidak ditemukan:", e)
    driver.quit()

def scroll_to_load_reviews():
    try:
        # Tunggu hingga elemen scrollable muncul
        scrollable_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="m6QErb DxyBCb kA9KIf dS8AEf XiKgde "]'))
        )

        last_height = 0  # Inisialisasi ketinggian sebelumnya
        while True:
            # Scroll ke bawah
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            time.sleep(5)  # Tunggu loading selesai

            # Dapatkan ketinggian kontainer setelah scroll
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

            # Jika ketinggian tidak berubah, artinya sudah sampai ujung
            if new_height == last_height:
                print("Reached the bottom of the reviews.")
                break

            # Perbarui ketinggian terakhir
            last_height = new_height
    except Exception as e:
        print(f"Error during scrolling: {e}")

# Scroll untuk memuat review
scroll_to_load_reviews()

# Ambil review
reviews = driver.find_elements(By.XPATH, '//div[@class="jftiEf fontBodyMedium "]')
stars = driver.find_elements(By.XPATH, '//span[@class="kvMYJc"]')

# Proses data
for review, star in zip(reviews, stars):
    try:
        username = review.find_element(By.XPATH, './/div[@class="d4r55 "]').text
    except:
        username = "Anonymous"
    try:
        review_text = review.find_element(By.XPATH, './/span[@class="wiI7pd"]').text
    except:
        review_text = "No review text"
    try:
        star_count = star.get_attribute("aria-label").split()[0]  # Ambil angka bintang
    except:
        star_count = "0"

    # Tambahkan data ke list
    data.append([username, review_text, star_count])

# Tutup browser
driver.quit()

Reached the bottom of the reviews.


In [35]:
# Buat DataFrame dari data
df_reviews = pd.DataFrame(data, columns=['Username', 'Review', 'Stars'])
df_reviews.to_csv('reviews.csv', index=False)

In [36]:
len(df_reviews)

360

In [37]:
df_reviews.head()

,Username,Review,Stars
0,Irfan fathurrahman,Safari Jateng di Batang merupakan salah satu d...,4
1,Ummu Nufaisah,"Taman satwa di pantai,tiket masuk dws 80rb,ana...",4
2,Uni Anila,"Kalau mau ke sini, lebih enak sih seharian sek...",4
3,Imam Santosa,Safari Beach Jateng\n\nMerupakan tempat wisata...,4
4,Citra Patriani,"Di rekomendasikan buat yg sayang dgn kluarga, ...",5
